In [1]:
pip install praw

  Obtaining dependency information for praw from https://files.pythonhosted.org/packages/81/6a/21bc058bcccbe03f6a0895bf1bd60c805f0c526aa4e9bfaac775ed0b299c/praw-7.7.1-py3-none-any.whl.metadata
  Obtaining dependency information for prawcore<3,>=2.1 from https://files.pythonhosted.org/packages/96/5c/8af904314e42d5401afcfaff69940dc448e974f80f7aa39b241a4fbf0cf1/prawcore-2.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for update-checker>=0.18 from https://files.pythonhosted.org/packages/0c/ba/8dd7fa5f0b1c6a8ac62f8f57f7e794160c1f86f31c6d0fb00f582372a3e4/update_checker-0.18.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 5.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
import praw
import pandas as pd

# Inizializza il Reddit client con le tue credenziali
reddit = praw.Reddit(client_id='OEgL2uPmYoDu61qnK_piSg',
                     client_secret='j7E3FBf5rGzy5vvVoQYuelBgVUlDLg',
                     user_agent='Massive_Advantage255')

# Lista per memorizzare i commenti
comments_list = []

# Lista degli ID dei post da cui ottenere i commenti
post_ids = ['1927x0u', '18l9u3b', '18ix1x6','18ixpku']  # Aggiungi qui gli ID dei post desiderati

# Itera sui post IDs
for post_id in post_ids:
    # Ottieni il post desiderato
    post = reddit.submission(id=post_id)
    
    # Estrai i commenti principali dal post
    post.comments.replace_more(limit=None)
    top_level_comments = list(post.comments)
    
    # Itera sui commenti principali e aggiungi le informazioni alla lista dei commenti
    for comment in top_level_comments:
        comments_list.append({
            'Post ID': post_id,
            'Comment ID': comment.id,
            'Text': comment.body,
            'Author': comment.author,
            'Score': comment.score,
            # Aggiungi altri attributi del commento se necessario
        })

# Converti la lista dei commenti in un DataFrame
comments_df = pd.DataFrame(comments_list)

# Visualizza il DataFrame
print(comments_df)


     Post ID Comment ID                                               Text  \
0    1927x0u    kh0paf5  Qualcuno disse che tutto ciò che sapeva su Chi...   
1    1927x0u    kh0y990  “Io non li volevo quei soldi, me li hanno adde...   
2    1927x0u    kh0zaw5  In che senso Alessandra Balocco? Non c'è nessu...   
3    1927x0u    kh0t6sd  Io non godo. Sarebbe stato meglio che non foss...   
4    1927x0u    kh0tnys  Non so quanto possa tenere botta una contestaz...   
..       ...        ...                                                ...   
240  18ixpku    kdvft0d                                 Buon Natale Chiaré   
241  18ixpku    kdw9nvq  A regà però a continuare a credere che la Ferr...   
242  18ixpku    kdwlkls  Tutti a dire : " la cosa che non capisco" e po...   
243  18ixpku    kdwlqzi           Che schifo che mi fanno lei e suo marito   
244  18ixpku    kdx8okl  Quanto mi stanno sullo scroto sti due non sapr...   

                   Author  Score  
0               RomoloJPN   

In [23]:
comments_list1 = comments_df['Text'].dropna().astype(str).tolist()

In [24]:
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Assuming red_df['Comment'] is your DataFrame column with comments

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("neuraly/bert-base-italian-cased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("neuraly/bert-base-italian-cased-sentiment")

def classify_comment(comment):
    # Tokenize and encode the comment for BERT
    input_ids = tokenizer.encode(comment, add_special_tokens=True, truncation=True, max_length=512)
    
    # Convert to tensor
    tensor = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension
    
    # Get the logits from the model
    with torch.no_grad():  # No need to compute gradients for inference
        logits = model(tensor)[0]
    
    # Apply softmax to calculate probabilities
    proba = nn.functional.softmax(logits, dim=1).squeeze(0)
    
    # Return the probabilities
    return proba.cpu().numpy()  # Use .cpu() to ensure tensor is on CPU when converting to numpy array

# Apply the function to each comment in the DataFrame column
# Use a list comprehension to apply 'classify_comment' to each comment in the list
results = [classify_comment(comment) for comment in comments_list1]

# Print some results
for index, (comment, sentiment) in enumerate(zip(comments_list1, results)):
    # Assuming 'sentiment' is a numpy array with the shape (3,), for negative, neutral, and positive probabilities
    negative, neutral, positive = sentiment
    print(f"Comment: {comment}\nSentiment: Negative={negative:.2f}, Neutral={neutral:.2f}, Positive={positive:.2f}\n")
    # Just print the first 10 results for brevity
    

Comment: Qualcuno disse che tutto ciò che sapeva su Chiara Ferragni lo sapeva contro la sua volontà. Spero che questa notizia sia perlomeno di suo gradimento
Sentiment: Negative=0.01, Neutral=0.82, Positive=0.17

Comment: “Io non li volevo quei soldi, me li hanno addebitati sul conto contro la mia volontà”
Sentiment: Negative=0.69, Neutral=0.31, Positive=0.00

Comment: In che senso Alessandra Balocco? Non c'è nessun signor Balocco? E allora chi è il vecchio delle pubblicità col nipote illegittimo?
Questa notizia mi ha stupito in senso negativo
Sentiment: Negative=0.10, Neutral=0.90, Positive=0.00

Comment: Io non godo. Sarebbe stato meglio che non fosse successo niente, e che Chiara Ferragni fosse veramente la persona che dona il 30% dei ricavi dei prodotti col suo marchio in beneficienza. 

Purtroppo non lo è.
Sentiment: Negative=1.00, Neutral=0.00, Positive=0.00

Comment: Non so quanto possa tenere botta una contestazione di truffa aggravata. Mi sembra un tantino tirata.

Finiranno p